In [1]:
# EfficientNet をインストール
!pip install -U git+https://github.com/qubvel/efficientnet

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-zyifxjvk
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-zyifxjvk
     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for efficientnet: filename=efficientnet-1.1.1-cp37-none-any.whl size=18421 sha256=4d941d9d393cafbc4c6fbf76402c4f9f6a3263383dd01a83f09101a908da9b65
  Stored in directory: /tmp/pip-ephem-wheel-cache-7e7r7eht/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet


In [2]:
# ライブラリの読み込み

import os
import sys
import random
import shutil
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
import tensorflow as tf

from skimage.io import imread
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.constraints import max_norm
from keras.applications.imagenet_utils import decode_predictions
from PIL import Image
from efficientnet.keras import EfficientNetB7
from efficientnet.keras import center_crop_and_resize, preprocess_input
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score


In [5]:
# ディレクトリ移動
%cd drive/MyDrive/mokumoku/dataset

/content/drive/MyDrive/mokumoku/dataset


In [ ]:
# 1回のみ実行
!unzip train.zip -d inputs/train
!unzip test.zip  -d inputs/test

In [6]:
# tsvファイルの読み込み
train_master = pd.read_csv('train_master.tsv', sep='\t')
label_master = pd.read_csv('label_master.tsv', sep='\t')

master = pd.merge(train_master, label_master, on='label_id', how='inner')
sample = pd.read_csv("sample_submit.csv", header=None, sep=",")

### 本コードを実行するためのディレクトリ構成
- カレントディレクトリ
    - inputs
        - train
        - test

mymoduleから実行するほうが良いですかね？

In [7]:
# このコードを実行するカレントディレクトリは"../inputs"を想定

DATADIR = "inputs"
training_data = []
test_data = []

def create_data(data):
    path = os.path.join(DATADIR, data)
    if data == 'train':
        for image_name, class_num, _ in master.values:
            img_array = img_to_array(load_img(os.path.join(path, image_name)))  # 画像読み込み            
            training_data.append([img_array, class_num])  # 画像データ、ラベル情報を追加
    if data == 'test':
        for image_name in sample[0].values:
            img_array = img_to_array(load_img(os.path.join(path, image_name))) # 画像読み込み            
            test_data.append(img_array)  # 画像データ、ラベル情報を追加
            
create_data('train')
create_data('test')

# 訓練データと検証データに8:2で分割
train_data, valid_data = train_test_split(training_data, shuffle=True, test_size=0.2, random_state=42)

# データセットの作成
X_train = []  # 画像データ
y_train = []  # ラベル情報

# データセット作成
for feature, label in train_data:
    X_train.append(feature)
    y_train.append(label)
# numpy配列に変換
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_valid = []  # 画像データ
y_valid = []  # ラベル情報

# データセット作成
for feature, label in valid_data:
    X_valid.append(feature)
    y_valid.append(label)
# numpy配列に変換
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)

# numpy配列に変換
test_data = np.asarray(test_data)

OSError: ignored

In [ ]:
import keras
import os
from efficientnet.keras import EfficientNetB7
from efficientnet.keras import center_crop_and_resize, preprocess_input
from keras.datasets import mnist
import numpy as np

epochs = 50
batch_size = 32


train_gen = ImageDataGenerator(
            featurewise_center = False,
            samplewise_center = False,
            featurewise_std_normalization = False,
            samplewise_std_normalization = False,
            zca_whitening = False,
            rotation_range = 30,
            width_shift_range = 0.3,
            height_shift_range = 0.3,
            horizontal_flip = True,
            vertical_flip = True,
        )

valid_gen = ImageDataGenerator(
            featurewise_center = False,
            samplewise_center = False,
            featurewise_std_normalization = False,
            samplewise_std_normalization = False,
            zca_whitening = False,
            rotation_range = 30,
            width_shift_range = 0.3,
            height_shift_range = 0.3,
            horizontal_flip = True,
            vertical_flip = True,
        )
y_train_categorical = keras.utils.to_categorical(y_train)
y_valid_categorical = keras.utils.to_categorical(y_valid)

# efficientnetに必要なところ。
# X_train_processed = preprocess_input(X_train)
# X_test_processed = preprocess_input(X_test)

n_classes = 20

# モデルの構築
INPUT_SHAPE = (32, 32, 3)
base_model = EfficientNetB7(input_shape=INPUT_SHAPE, weights='imagenet', include_top=False)
x = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation='softmax')(x)
model = keras.models.Model(inputs=[base_model.input], outputs=[output])


# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
train_gen.fit(X_train)
valid_gen.fit(X_valid)

model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
tb_cb = keras.callbacks.TensorBoard(log_dir='output', histogram_freq=1)
# fits the model on batches with real-time data augmentation:
history = model.fit_generator(train_gen.flow(X_train, y_train_categorical, batch_size=batch_size),
                    epochs=epochs,
                    steps_per_epoch=X_train.shape[0] // batchi_size,
                    validation_data=valid_gen.flow(X_valid, y_valid_categorical, batch_size=batchi_size),
                    validation_steps=X_valid.shape[0] // batchi_size,
                    verbose=1,
                    )

In [ ]:
print(model.evaluate(X_valid, y_valid_categorical, verbose=0))

predictions = model.predict(test_data)
# [0.9517145752906799, 0.7093999981880188]

In [ ]:
file_name = sample[0]
df = pd.DataFrame(predictions)
df_concat = pd.concat([file_name, df],axis=1)
df_concat.head()
df_concat.to_csv('submit_002.csv',index = False, header=None)